In [1]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris


# Import PySwarms
import pyswarms as ps

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
data = load_iris()

In [3]:
# Store the features as X and the labels as y
X = data.data
y = data.target

In [4]:
n_inputs = 4
n_hidden = 20
n_classes = 3

num_samples = 150

In [5]:
def logits_function(p):
    """ Calculate roll-back the weights and biases

    Inputs
    ------
    p: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    numpy.ndarray of logits for layer 2

    """
    # Roll-back the weights and biases
    W1 = p[0:80].reshape((n_inputs,n_hidden))
    b1 = p[80:100].reshape((n_hidden,))
    W2 = p[100:160].reshape((n_hidden,n_classes))
    b2 = p[160:163].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    logits = a1.dot(W2) + b2 # Pre-activation in Layer 2
    return logits          # Logits for Layer 2

In [6]:
# Forward propagation
def forward_prop(params):
    """Forward propagation as objective function

    This computes for the forward propagation of the neural network, as
    well as the loss.

    Inputs
    ------
    params: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    float
        The computed negative log-likelihood loss given the parameters
    """

    logits = logits_function(params)

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood

    corect_logprobs = -np.log(probs[range(num_samples), y])
    loss = np.sum(corect_logprobs) / num_samples

    return loss

In [7]:
def f(x):
    """Higher-level method to do forward_prop in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

In [8]:
%%time
kappa = 0.90
phi1 = 2.05
phi2 = 2.05
phi = phi1 + phi2
chi = 2*kappa / abs(2-phi-np.sqrt(phi**2-4*phi))

# Initialize swarm
options = {'c1': chi*phi1, 'c2': chi*phi2, 'w':chi}

# Call instance of PSO
dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(f, iters=1000)

2022-05-05 16:24:47,259 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 1.3465617890968198, 'c2': 1.3465617890968198, 'w': 0.6568594093155219}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=9.18e-6
2022-05-05 16:25:06,596 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 9.181355288612518e-06, best pos: [-8.76651694e-02  8.09388523e-01 -4.07902049e-01  1.59429497e+00
 -1.01556566e+00 -5.15788010e-01 -4.52734455e-02  3.46425589e+00
 -2.74210233e-01 -1.39271107e+00  3.02745419e+00  6.27815580e-01
  7.89852228e-01 -2.74413640e-01  1.29346991e+00 -6.77204426e-01
  1.67377103e+01  1.71314116e+00  1.53600293e+00  1.97312329e+00
  8.72072162e-01  2.58501688e-01  3.40776800e-01  1.33485917e+00
  1.59860606e+00 -1.41068659e-01 -1.50144018e+00  2.49284853e+00
 -1.27774837e+00  1.69622346e+00  6.12890037e-01  3.03174750e-01
  3.86105022e+00  1.55348159e+00  1.14212968e+00  1.29296979e+00
 -1.18528063e-02 -5.27297549e-01 -3.1738

CPU times: total: 4 s
Wall time: 19.3 s


In [9]:
def predict(pos):
    """
    Use the trained weights to perform class predictions.

    Inputs
    ------
    pos: numpy.ndarray
        Position matrix found by the swarm. Will be rolled
        into weights and biases.
    """
    logits = logits_function(pos)
    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [10]:
(predict(pos) == y).mean()

1.0